# 1) Titolo e obiettivi
Lezione 39: AI nel mondo reale - soglie, rischio e fallback umano.

---

## Mappa concettuale della lezione

```
AI IN PRODUZIONE - PIPELINE DECISIONALE
=========================================

    +------------------+
    |  INPUT (caso)    |
    |  ticket, request |
    +--------+---------+
             |
             v
    +------------------+
    |  MODELLO ML      |
    |  score/confidenza|
    +--------+---------+
             |
             v
    +------------------+
    |  POLICY ENGINE   |
    |  soglie, regole  |
    +--------+---------+
             |
    +--------+---------+--------+
    |                           |
    v                           v
+----------+            +------------+
|   AUTO   |            |   HUMAN    |
| decision |            |  fallback  |
+----------+            +------------+


SOGLIE E COSTI - TRADE-OFF
===========================

    SOGLIA BASSA                    SOGLIA ALTA
    ============                    ============
    Piu' automazione                Meno automazione
    Piu' falsi positivi             Meno falsi positivi
    Piu' rischio                    Piu' costo umano
    Piu' copertura                  Meno copertura


MONITORAGGIO CONTINUO
=====================

    Week 1    Week 2    Week 3    Week 4
    ------    ------    ------    ------
    [=====]   [=====]   [====]    [===]     <- Score medio
    
    DRIFT DETECTATO: score medio scende -> ALERTA!
```

---

## Obiettivi didattici

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Progettare policy di automazione basate su confidenza | Operativo |
| 2 | Calcolare copertura e tasso di errore | Analitico |
| 3 | Implementare fallback umano per casi incerti | Pratico |
| 4 | Monitorare drift degli score nel tempo | Avanzato |
| 5 | Mantenere audit log per compliance | Governance |
| 6 | Bilanciare costo errore vs costo revisione umana | Strategico |

---

## Concetti chiave

> **Copertura**: percentuale di casi gestiti automaticamente dal modello. Copertura alta = piu' efficienza ma piu' rischio.

> **Fallback umano**: instradamento dei casi a bassa confidenza verso revisori umani per ridurre errori critici.

> **Drift**: cambiamento nella distribuzione degli input o degli score nel tempo, segnale che il modello potrebbe degradare.

---

## Matrice costo-rischio

```
                    COSTO ERRORE BASSO    COSTO ERRORE ALTO
                    ==================    =================
CONFIDENZA ALTA     ✓ Automatizza         ✓ Automatizza
                      (basso rischio)       (accettabile)

CONFIDENZA BASSA    ✓ Automatizza         ✗ FALLBACK UMANO
                      (errore costa poco)   (errore costa molto!)


ESEMPIO PRATICO:
----------------
- Email spam: errore costa poco → soglia bassa OK
- Frode bancaria: errore costa molto → soglia alta + review
- Diagnosi medica: errore critico → sempre review umano
```

---

## Cosa useremo
- NumPy/Pandas per simulazioni
- Funzioni di policy con soglie configurabili
- Metriche di copertura e falsi positivi
- Audit logging per tracciabilita'

## Prerequisiti
- Precision/Recall (Lezione 17)
- Confusion matrix
- Concetto di threshold optimization


# 2) Teoria concettuale
- In produzione i modelli espongono score/valori di confidenza; serve scegliere soglie in base al costo degli errori.
- Fallback umano: instradare i casi incerti a revisori per ridurre il rischio.
- Monitoraggio: drift degli score e auditing delle decisioni.


# 3) Schema mentale / mappa decisionale
1. Definisci classi a rischio e le soglie per automazione.
2. Calcola copertura automatica (quanti casi gestiti dal modello).
3. Monitora drift degli score; registra audit log.
4. Aggiorna soglie/policy in base a costi ed evidenze.


# 4) Sezione dimostrativa
Demo: simulazione di score su ticket, policy soglia, copertura, drift e log di audit.


In [ ]:
# Setup simulazione
import numpy as np
import pandas as pd
np.random.seed(42)


In [ ]:
# Simuliamo score di confidenza per 200 ticket con label vera (1=alto rischio)
n = 200
scores = np.clip(np.random.normal(0.6, 0.15, n), 0, 1)
labels = (np.random.rand(n) > 0.7).astype(int)
df = pd.DataFrame({'score': scores, 'label': labels})
print(df.head())
assert not df.isna().any().any()


In [ ]:
# Policy: se score >= soglia -> auto; altrimenti escalation umana
threshold = 0.65

def apply_policy(row):
    decision = 'auto' if row['score'] >= threshold else 'human'
    return decision

df['decision'] = df.apply(apply_policy, axis=1)
coverage = (df['decision'] == 'auto').mean()
print(f"Copertura automatica: {coverage:.2%}")


In [ ]:
# Stimiamo rischio: falsi positivi tra i casi auto (se label=0 ma auto)
fps = ((df['decision']=='auto') & (df['label']==0)).mean()
print(f"Falsi positivi (sul totale casi): {fps:.2%}")


### Osservazioni
- Aumentare la soglia riduce falsi positivi ma riduce copertura automatica.
- Serve scegliere la soglia in base al costo errore vs costo umano.


# 5) Esercizi svolti (step-by-step)
## Esercizio 39.1 - Drift semplice


In [ ]:
# Esercizio 39.1: drift dello score medio
score_new = np.clip(np.random.normal(0.45, 0.15, n), 0, 1)
mean_old = df['score'].mean()
mean_new = score_new.mean()
print(f"Mean score vecchio: {mean_old:.3f}, nuovo: {mean_new:.3f}")
if abs(mean_new - mean_old) > 0.05:
    print("Possibile drift: rivalutare la soglia")


## Esercizio 39.2 - Audit log


In [ ]:
# Esercizio 39.2: audit log
from datetime import datetime

df['audit'] = df.apply(lambda r: {
    'ts': datetime.utcnow().isoformat(),
    'score': r['score'],
    'decision': r['decision']
}, axis=1)
print(df[['audit']].head())


## Esercizio 39.3 - Policy class-specific


In [ ]:
# Esercizio 39.3: soglia diversa per classe ad alto costo
costly_class_prob = np.random.rand(n)
threshold_high_cost = 0.75

decisions = []
for s, cost_prob in zip(scores, costly_class_prob):
    if cost_prob > 0.8 and s < threshold_high_cost:
        decisions.append('human')
    else:
        decisions.append('auto' if s >= threshold else 'human')

print(pd.Series(decisions).value_counts(normalize=True))


# 6) Conclusione operativa - Bignami AI in Produzione

---

## I 5 Take-Home Messages

| # | Concetto | Perche' conta |
|---|----------|---------------|
| 1 | **Soglia = business decision, non tecnica** | Dipende da costo errore e costo review |
| 2 | **Copertura e accuracy sono in trade-off** | Piu' automazione = piu' rischio errori |
| 3 | **Classi diverse richiedono policy diverse** | Alto rischio → soglia piu' alta |
| 4 | **Drift detection e' obbligatorio** | Modello degrada se dati cambiano |
| 5 | **Audit log per compliance e debug** | Devi poter spiegare ogni decisione |

---

## Policy decision framework

```
STEP 1: CLASSIFICAZIONE RISCHIO
================================

    Input ──► Risk Class ──► Policy Selection
    
    - Risk LOW:   threshold = 0.5
    - Risk MED:   threshold = 0.7
    - Risk HIGH:  threshold = 0.9 + human review


STEP 2: DECISION ENGINE
=======================

    IF score >= threshold:
        decision = "AUTO"
        action = model_prediction
    ELSE:
        decision = "HUMAN"
        action = route_to_queue


STEP 3: MONITORING
==================

    log = {
        "timestamp": now(),
        "input_id": case_id,
        "score": model_score,
        "threshold": used_threshold,
        "decision": "AUTO" | "HUMAN",
        "outcome": actual_result  # post-hoc
    }
```

---

## Metriche operative

| Metrica | Formula | Interpretazione |
|---------|---------|-----------------|
| Copertura | auto / total | % casi automatizzati |
| FP rate (auto) | FP_auto / auto | Errori su casi automatizzati |
| Costo totale | FP×costo_errore + human×costo_review | Obiettivo: minimizzare |
| Drift score | |mean(t) - mean(t-1)| / std | Alerta se > threshold |

---

## Template policy engine

```python
import pandas as pd
import numpy as np
from datetime import datetime

class PolicyEngine:
    def __init__(self, default_threshold=0.7, high_risk_threshold=0.9):
        self.default_threshold = default_threshold
        self.high_risk_threshold = high_risk_threshold
        self.audit_log = []
    
    def get_threshold(self, risk_class):
        """Threshold dinamico per classe di rischio"""
        thresholds = {
            'low': 0.5,
            'medium': self.default_threshold,
            'high': self.high_risk_threshold
        }
        return thresholds.get(risk_class, self.default_threshold)
    
    def decide(self, case_id, score, risk_class='medium'):
        """Decision engine con logging"""
        threshold = self.get_threshold(risk_class)
        decision = 'AUTO' if score >= threshold else 'HUMAN'
        
        # Audit logging
        self.audit_log.append({
            'timestamp': datetime.now(),
            'case_id': case_id,
            'score': score,
            'risk_class': risk_class,
            'threshold': threshold,
            'decision': decision
        })
        return decision
    
    def get_coverage(self):
        """Calcola copertura automatica"""
        if not self.audit_log:
            return 0.0
        df = pd.DataFrame(self.audit_log)
        return (df['decision'] == 'AUTO').mean()
    
    def detect_drift(self, window=7, threshold=0.1):
        """Rileva drift degli score"""
        if len(self.audit_log) < 2 * window:
            return False, 0.0
        df = pd.DataFrame(self.audit_log)
        recent = df['score'].tail(window).mean()
        previous = df['score'].iloc[-2*window:-window].mean()
        drift = abs(recent - previous)
        return drift > threshold, drift
    
    def export_audit(self):
        """Esporta log per compliance"""
        return pd.DataFrame(self.audit_log)

# Uso
engine = PolicyEngine(default_threshold=0.7)
for i, (score, risk) in enumerate(zip(scores, risk_classes)):
    decision = engine.decide(f"case_{i}", score, risk)
print(f"Copertura: {engine.get_coverage():.1%}")
```

---

## Errori comuni e soluzioni

| Errore | Conseguenza | Soluzione |
|--------|-------------|-----------|
| Soglia fissa per tutti i casi | Troppi errori su casi ad alto rischio | Policy class-based |
| Nessun monitoraggio drift | Modello degrada silenziosamente | Alert su score distribution |
| Mancanza di audit log | Non-compliance, no debugging | Log ogni decisione |
| Ignorare costo review | Sottostima costo totale | Include human cost in optimization |

---

## Checklist deployment AI

```
PRE-DEPLOYMENT
☐ Definite classi di rischio
☐ Soglie calibrate su costi
☐ Fallback umano configurato
☐ Audit logging implementato

POST-DEPLOYMENT
☐ Monitoraggio drift attivo
☐ Alert configurati
☐ Review periodica decisioni
☐ Feedback loop per retraining
```


# 7) Checklist di fine lezione
- [ ] Ho definito soglia(e) basate su costi.
- [ ] Ho misurato copertura automatica e falsi positivi.
- [ ] Ho considerato classi ad alto costo con policy dedicate.
- [ ] Ho previsto monitoraggio drift degli score.
- [ ] Ho registrato decisioni in un audit log.

Glossario
- Copertura: quota di casi gestiti automaticamente.
- Falso positivo: caso auto ma label negativa.
- Drift: cambiamento statistico degli score/dati nel tempo.
- Audit log: registro delle decisioni per controllo.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-XX | Struttura iniziale 8 sezioni |
| 2.0 | 2024-12-XX | Espansione completa AI in Produzione |
| 2.1 | - | Pipeline decisionale ASCII completa |
| 2.2 | - | Trade-off soglie visualizzato |
| 2.3 | - | Matrice costo-rischio per decisioni |
| 2.4 | - | Classe PolicyEngine completa |
| 2.5 | - | Drift detection implementata |
| 2.6 | - | Checklist pre/post deployment |

---

## Note di versione

**v2.0 - Espansione didattica completa**
- Focus su aspetti operativi del deployment AI
- Policy engine come pattern riutilizzabile
- Emphasis su audit e compliance
- Drift detection come best practice
- Preparazione per MLOps e monitoring avanzato

**Dipendenze didattiche**
- Richiede: Lezione 17 (metriche), threshold optimization
- Prepara: Concetti MLOps, CI/CD per modelli, A/B testing
